In [1]:
import pandas as pd
import numpy as np
import sys
import multiprocessing
import time

sys.path.append(('/home/ak/Research/PaperCode/stylised_facts'))
sys.path.append('/home/ak/Research/PaperCode/stylised_facts')
import stylised_facts_data_utilities as sfd_utils
import lob_for_futures as lobfut
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from multiprocessing import Pool, freeze_support
import pickle
from multiprocessing import Pool

scaler = MinMaxScaler()
standard_scaler = StandardScaler()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sys.path.insert(0, '/directory/tothe/handshakefile/')
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')
## data files


In [19]:
#laptop_OS_folder = '/media/ak/T71/FuturesDataSemiProcessed'

laptop_OS_folder = '/media/ak/T7/FuturesDataSemiProcessed'
LaCie_ProcessedData = '/media/ak/LaCie/ProcessedSampledData/'
# returns_data = '/media/ak/T7/August11th2022Experiments/Returns/'
t7 = '/media/ak/T7/'
t7_folder = os.path.join(t7, 'FuturesDataSemiProcessed')
# june_ext = os.path.join(t7, 'June4th2022Experiments')
# returns_data = [f for f in os.listdir(june_ext) if '_returns' in f]
experimentsLocation = '/media/ak/T7/August11th2022Experiments/'
# here i start with RX1 to do all the experiments in one go
symbols = [f for f in os.listdir(laptop_OS_folder) if str('Returns') not in f]

In [22]:
print(symbols)

['VIX', 'FB1', 'TU1', 'RX1', 'OE1', 'TY1', 'FV1', 'JB1', 'DU1', 'KE1', 'US1', 'YM1', 'XM1', 'VXX', 'G_1']


In [3]:
## checking folders in the small laptop

os.path.isdir(experimentsLocation)

True

In [4]:


# symbol_test_folder = os.path.join(laptop_OS_folder, symbols[0]) rx_folder = os.path.join(laptop_OS_folder,
# 'RX1') du_folder = os.path.join(t7_folder,  'DU1')
# , 'FB1') make this a bit more dynamic to take any function in here files = os.listdir(du_folder)


def produce_experiment_data(chosen_df):
    """
    # this is somewhat unit-tested in the August 7th 2022 notebook
    # function to produce dataframes for experiments
    # input: chosen df - this is a dataframe that we apply the microstructure features and the vol estimation features
    # output: experiment - dataframe with the features we want experiments for
    # written in August 2022
    # re-write it as part of the _init_ file

    """
    chosen_df_micro = lobfut.apply_micro_structure_features(chosen_df)  # get micro structure df
    vol_class = lobfut.volatilityEstimation(chosen_df)  # get the vol class

    # features I need: micro_price changes / vols /skews /etc:

    experiment_df = chosen_df_micro.loc[:, ['micro_price', 'price_imbalance',
                                            'pct_change_micro_price', 'weighted_activity_spread', ]]
    experiment_df['GK_vol'] = pd.Series(
        list(vol_class.garmanKlass(5)))  # get a sample of Garman - Klass resampled for 5 clicks
    experiment_df['arrival_rates'] = pd.Series(
        vol_class.arrival_rates().reshape(vol_class.arrival_rates().shape[0], )).replace([np.inf, -np.inf],
                                                                                         0).fillna(0)
    X = experiment_df.pct_change_micro_price.replace([np.inf, -np.inf], 0).values.reshape(-1, 1)
    norm_scaler = StandardScaler().fit(X)  # normalised scaling by mean and std
    min_max_scaler = MinMaxScaler().fit(X)  # min max scaling
    try:
        experiment_df['returns_normalised'] = norm_scaler.transform(
            X)  # use this format to get rid of the prior issues
        experiment_df['returns_mix_max'] = min_max_scaler.transform(X)  # get both issues of
        rs, rk = vol_class.realised_skewness_kurtosis()  # get skew/kurt

        experiment_df['skew'] = pd.Series(list(rs))
        experiment_df['kurt'] = pd.Series(list(rk))

        experiment_df['median_traded_volume'] = chosen_df_micro[['total_traded_volume_open',
                                                                 'total_traded_volume_high',
                                                                 'total_traded_volume_low',
                                                                 'total_traded_volume_close']].quantile(0.5, axis=1)
        experiment_df['jumps_test'] = pd.Series(vol_class.jumps_test(rollingWindow=5, sampling_param=0))
        experiment_df['relz_var'] = pd.Series(vol_class.realised_variance(rollingWindow=5))
        experiment_df['trip_quart'] = pd.Series(vol_class.tripower_quarticity(rollingWindow=5, sampling_param=0))

        experiment_df = experiment_df.replace([np.inf, -np.inf], 0).fillna(0)  # final clean up
    except ValueError:
        print("error")
        pass
    experiment_df = experiment_df.replace([np.inf, -np.inf], 0).fillna(0)  # final clean up

    return experiment_df



In [8]:
symbol ='KE1'
symbol_folder_path = os.path.join(t7_folder, str(symbol))
print(symbol_folder_path)
all_files = os.listdir(symbol_folder_path)
all_files

/media/ak/T7/FuturesDataSemiProcessed/KE1


['20180718.pkl',
 '20180730.pkl',
 '20180629.pkl',
 '20180727.pkl',
 '20181023.pkl',
 '20180607.pkl',
 '20180424.pkl',
 '20180611.pkl',
 '20180416.pkl',
 '20181015.pkl',
 '20180502.pkl',
 '20180904.pkl',
 '20180530.pkl',
 '20180704.pkl',
 '20180927.pkl',
 '20180831.pkl',
 '20180614.pkl',
 '20180815.pkl',
 '20180711.pkl',
 '20180613.pkl',
 '20180601.pkl',
 '20180528.pkl',
 '20181024.pkl',
 '20180820.pkl',
 '20180628.pkl',
 '20180719.pkl',
 '20181019.pkl',
 '20180705.pkl',
 '20180604.pkl',
 '20181016.pkl',
 '20180806.pkl',
 '20180627.pkl',
 '20181010.pkl',
 '20180725.pkl',
 '20180622.pkl',
 '20180803.pkl',
 '20180425.pkl',
 '20180903.pkl',
 '20180529.pkl',
 '20181005.pkl',
 '20180516.pkl',
 '20180615.pkl',
 '20180829.pkl',
 '20180814.pkl',
 '20180608.pkl',
 '20180807.pkl',
 '20180919.pkl',
 '20180723.pkl',
 '20180621.pkl',
 '20180619.pkl',
 '20180626.pkl',
 '20180822.pkl',
 '20180813.pkl',
 '20180821.pkl',
 '20181002.pkl',
 '20180720.pkl',
 '20180920.pkl',
 '20180606.pkl',
 '20180907.pkl

In [9]:
symbol_folder_path = os.path.join(t7_folder, str(symbol))
all_files = os.listdir(symbol_folder_path)
files = [f for f in all_files if str('Returns_') not in f]

In [23]:
files_idx_ =20
choice_bar = 'volume'  # change this
date_idx = files[files_idx_].split(".")[0]
print(date_idx)

20180601


In [15]:
# symbol_folder_path = os.path.join(t7_folder, str(symbol))
#         all_files = os.listdir(symbol_folder_path)
#         files = [f for f in all_files if str('Returns_') not in f]

files_idx = 10
choice_bar = 'volume'  # change this
date_idx = files[files_idx_].split(".")[0]
print(date_idx)

idx_file_path = os.path.join(symbol_folder_path,
                             files[files_idx_])  # the input here needs to be dynamic not du_folder or rx_folder
choice_df = pd.read_pickle(idx_file_path)[date_idx][choice_bar]

        #

20180502


In [16]:
choice_df

,BidSize_open,BidSize_high,BidSize_low,BidSize_close,QuoteTime_open,QuoteTime_high,QuoteTime_low,QuoteTime_close,type_x_open,type_x_high,type_x_low,type_x_close,BestBid_open,BestBid_high,BestBid_low,BestBid_close,TradeTime_open,TradeTime_high,TradeTime_low,TradeTime_close,AskSize_open,AskSize_high,AskSize_low,AskSize_close,time_y_open,time_y_high,time_y_low,time_y_close,type_y_open,type_y_high,type_y_low,type_y_close,BestAsk_open,BestAsk_high,BestAsk_low,BestAsk_close,TradeSize_open,TradeSize_high,TradeSize_low,TradeSize_close,type_open,type_high,type_low,type_close,TradePrice_open,TradePrice_high,TradePrice_low,TradePrice_close,TimeStamp_open,TimeStamp_high,TimeStamp_low,TimeStamp_close,TradeVolume_open,TradeVolume_high,TradeVolume_low,TradeVolume_close,total_traded_volume_open,total_traded_volume_high,total_traded_volume_low,total_traded_volume_close,milliSeconds_open,milliSeconds_high,milliSeconds_low,milliSeconds_close,dollar_traded_volume_open,dollar_traded_volume_high,dollar_traded_volume_low,dollar_traded_volume_close,timeStampIdx_open,timeStampIdx_high,timeStampIdx_low,timeStampIdx_close,micro_price_open,micro_price_high,micro_price_low,micro_price_close
volume_bucket,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16.0,368,368,368,368,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,BID,BID,BID,BID,108.15,108.15,108.15,108.15,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,864,864,864,864,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,ASK,ASK,ASK,ASK,108.16,108.16,108.16,108.16,833,833,833,833,TRADE,TRADE,TRADE,TRADE,108.15,108.15,108.15,108.15,00:00:01,00:00:01,00:00:01,00:00:01,833,833,833,833,833,833,833,833,1000,1000,1000,1000,90088.95,90088.95,90088.95,90088.95,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,108.157013,108.157013,108.157013,108.157013
24.0,371,371,371,371,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,BID,BID,BID,BID,108.15,108.15,108.15,108.15,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,864,864,864,864,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,ASK,ASK,ASK,ASK,108.16,108.16,108.16,108.16,833,833,833,833,TRADE,TRADE,TRADE,TRADE,108.15,108.15,108.15,108.15,00:00:01,00:00:01,00:00:01,00:00:01,833,833,833,833,833,833,833,833,1000,1000,1000,1000,90088.95,90088.95,90088.95,90088.95,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,2018-07-18 00:00:01,108.156996,108.156996,108.156996,108.156996
33.0,389,389,389,389,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,BID,BID,BID,BID,108.15,108.15,108.15,108.15,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,867,867,867,867,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,ASK,ASK,ASK,ASK,108.16,108.16,108.16,108.16,833,833,833,833,TRADE,TRADE,TRADE,TRADE,108.15,108.15,108.15,108.15,00:00:02,00:00:02,00:00:02,00:00:02,833,833,833,833,833,833,833,833,2000,2000,2000,2000,90088.95,90088.95,90088.95,90088.95,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,108.156903,108.156903,108.156903,108.156903
41.0,390,390,390,390,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,BID,BID,BID,BID,108.15,108.15,108.15,108.15,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,867,867,867,867,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,ASK,ASK,ASK,ASK,108.16,108.16,108.16,108.16,833,833,833,833,TRADE,TRADE,TRADE,TRADE,108.15,108.15,108.15,108.15,00:00:02,00:00:02,00:00:02,00:00:02,833,833,833,833,833,833,833,833,2000,2000,2000,2000,90088.95,90088.95,90088.95,90088.95,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,2018-07-18 00:00:02,108.156897,108.156897,108.156897,108.156897
49.0,